In [2]:
import pandas as pd
import os

# Define the path to the Excel file
# Make sure the file name matches exactly what is in your folder
FILE_PATH = "data/players24_25.xlsx"

# Check if the file exists before trying to read it
if not os.path.exists(FILE_PATH):
    print(f"ERROR: Could not find file at path: {FILE_PATH}")
    print("Please check that the file name is correct and located in the 'data' folder.")
else:
    print(f"File found: {FILE_PATH}")
    
    # Read the Excel file
    try:
        df = pd.read_excel(FILE_PATH)
        print("Data loaded successfully.")
        
        # Display dataset size
        print(f"Number of players: {len(df)}")
        print(f"Number of columns: {len(df.columns)}")
        
        # Show the first few rows for inspection
        display(df.head())
        
        # Print all column names to select the relevant features
        print("--- ALL COLUMNS ---")
        print(list(df.columns))
        
    except Exception as e:
        print(f"An error occurred while reading the Excel file: {e}")

File found: data/players24_25.xlsx
Data loaded successfully.
Number of players: 2854
Number of columns: 37


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Gls_90,Ast_90,G+A_90,G-PK_90,G+A-PK_90,xG_90,xAG_90,xG+xAG_90,npxG_90,npxG+xAG_90
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,24.0,2000.0,3,1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
1,2,Max Aarons,eng ENG,"DF,MF",Valencia,es La Liga,24.0,2000.0,4,1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.02,0.02,0.00,0.02
2,3,Rodrigo Abajas,es ESP,DF,Valencia,es La Liga,21.0,2003.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.10,0.00,0.10,0.10,0.10
3,4,James Abankwah,ie IRL,"DF,MF",Udinese,it Serie A,20.0,2004.0,6,0,...,0.0,0.0,0.0,0.0,0.0,0.06,0.00,0.06,0.06,0.06
4,5,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18.0,2006.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00


--- ALL COLUMNS ---
['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls_90', 'Ast_90', 'G+A_90', 'G-PK_90', 'G+A-PK_90', 'xG_90', 'xAG_90', 'xG+xAG_90', 'npxG_90', 'npxG+xAG_90']


In [3]:
# --- CONFIGURATION ---
MIN_MINUTES = 180  # Only keep players who played at least ~2 full matches
# ---------------------

# 1. Filter out players with too little playing time to avoid statistical noise
df_clean = df[df['Min'] >= MIN_MINUTES].copy()

# 2. Fill missing values (NaN) with 0 to prevent errors
df_clean = df_clean.fillna(0)

print(f"Players remaining after filtering: {len(df_clean)}")

def create_scouting_report(row):
    """
    Converts a row of data into a descriptive string.
    """
    # Basic Info
    text = (
        f"Scouting Report for {row['Player']}. "
        f"Team: {row['Squad']}. "
        f"Position: {row['Pos']}. "
        f"Age: {row['Age']}. "
        f"Nation: {row['Nation']}. "
    )
    
    # Playing Time
    text += f"Matches Played: {row['MP']}. Minutes: {row['Min']}. "
    
    # Offensive Output
    text += (
        f"Goals: {row['Gls']}. "
        f"Assists: {row['Ast']}. "
        f"Non-Penalty xG (npxG): {row['npxG']}. "
        f"Expected Assists (xAG): {row['xAG']}. "
    )
    
    # Progression / Playstyle (The 'Moneyball' stats)
    # PrgC = Progressive Carries (dribbling forward)
    # PrgP = Progressive Passes (passing forward)
    text += (
        f"Progressive Carries: {row['PrgC']}. "
        f"Progressive Passes: {row['PrgP']}. "
    )
    
    return text

# 3. Apply the function to every row
df_clean['scouting_report'] = df_clean.apply(create_scouting_report, axis=1)

# 4. Display an example to verify it looks correct
print("\n--- EXAMPLE REPORT ---")
print(df_clean['scouting_report'].iloc[0])

Players remaining after filtering: 2328

--- EXAMPLE REPORT ---
Scouting Report for Yunis Abdelhamid. Team: Saint-Étienne. Position: DF. Age: 36.0. Nation: ma MAR. Matches Played: 16. Minutes: 1033. Goals: 0. Assists: 0. Non-Penalty xG (npxG): 0.2. Expected Assists (xAG): 0.1. Progressive Carries: 4. Progressive Passes: 22. 


In [4]:
import os
import shutil
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document 
from dotenv import load_dotenv

# 1. Load environment variables
load_dotenv()

if not os.getenv("GOOGLE_API_KEY"):
    print("ERROR: GOOGLE_API_KEY not found. Please check your .env file.")
else:
    print("Google API Key loaded successfully.")

# --- CONFIGURATION ---
CHROMA_PATH = "chroma_db"
# ---------------------

# Close existing db connection if it exists
if 'db' in dir() and db is not None:
    try:
        del db
        print("Closed existing database connection.")
    except:
        pass

# DELETE existing database to avoid duplicates
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)
    print(f"Deleted existing database at '{CHROMA_PATH}'")

# 2. Prepare documents
documents = []
print("Creating documents from player data...")

for index, row in df_clean.iterrows():
    content = row['scouting_report']
    meta = {
        "player_name": row['Player'],
        "team": row['Squad'],
        "position": row['Pos'],
        "nation": row['Nation'],
        "age": int(row['Age']) if pd.notnull(row['Age']) and row['Age'] != 0 else 0
    }
    doc = Document(page_content=content, metadata=meta)
    documents.append(doc)

print(f"Prepared {len(documents)} documents.")

# 3. Create Vector Database using GEMINI EMBEDDINGS
print("Generating embeddings with Gemini (this may take a minute)...")

embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

db = Chroma.from_documents(
    documents=documents, 
    embedding=embedding_function, 
    persist_directory=CHROMA_PATH
)

print(f"SUCCESS: Database saved to folder '{CHROMA_PATH}' with {len(documents)} players!")

Google API Key loaded successfully.
Deleted existing database at 'chroma_db'
Creating documents from player data...
Prepared 2328 documents.
Generating embeddings with Gemini (this may take a minute)...
SUCCESS: Database saved to folder 'chroma_db' with 2328 players!


In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

# 1. Setup the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# 2. Create a Custom Prompt
template = """You are an expert football scout for a top European club. 
Use the following pieces of context (player stats) to answer the question at the end.

Rules:
1. Only recommend players found in the context.
2. If you don't know the answer or can't find a suitable player, say so.
3. Always justify your recommendation with stats (Goals, xG, Progressive Carries, etc).
4. Keep the tone professional and analytical.

Context:
{context}

Question: {question}

Helpful Answer:"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 3. Function to query with position filtering
def ask_scout(question, position_filter=None):
    """
    Ask the scout AI a question.
    position_filter: 'FW', 'MF', 'DF', or None for all positions
    """
    # Get more results initially, then filter
    all_docs = db.similarity_search(question, k=50)
    
    # Filter by position if specified
    if position_filter:
        docs = [d for d in all_docs if position_filter in d.metadata.get('position', '')][:10]
    else:
        # Auto-detect position from question
        q = question.lower()
        if any(word in q for word in ['forward', 'striker', 'attacker']):
            docs = [d for d in all_docs if 'FW' in d.metadata.get('position', '')][:10]
        elif any(word in q for word in ['midfielder', 'midfield', 'playmaker']):
            docs = [d for d in all_docs if 'MF' in d.metadata.get('position', '')][:10]
        elif any(word in q for word in ['defender', 'defense', 'centre-back', 'fullback']):
            docs = [d for d in all_docs if 'DF' in d.metadata.get('position', '')][:10]
        else:
            docs = all_docs[:10]
    
    print("--- Retrieved Players ---")
    for doc in docs:
        print(f"- {doc.metadata['player_name']} ({doc.metadata['position']}, Age: {doc.metadata['age']})")
    
    context = format_docs(docs)
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"context": context, "question": question})
    
    print("\n--- Scout AI Response ---")
    print(response)
    return response

print("Scout AI is ready!")
print("Use: ask_scout('your question')")
print("Or:  ask_scout('your question', position_filter='FW')  # FW, MF, or DF")

Scout AI is ready!
Use: ask_scout('your question')
Or:  ask_scout('your question', position_filter='FW')  # FW, MF, or DF


In [ ]:
ask_scout("Who are the best forwards with high goals and good progressive carries?")

--- Retrieved Players ---
- Connor Metcalfe (MF,FW, Age: 24)
- Anastasios Douvikas (FW, Age: 24)
- Joel Pohjanpalo (FW, Age: 29)
- Richarlison (FW, Age: 27)
- Jamie Gittens (FW,MF, Age: 19)
- Jacob Bruun Larsen (FW,MF, Age: 25)
- Anastasios Douvikas (FW, Age: 24)
- Giorgos Masouras (FW,MF, Age: 30)
- Reiss Nelson (FW, Age: 24)
- Andrea Belotti (FW, Age: 30)

--- Scout AI Response ---
Based on the provided scouting reports, the following forwards stand out in terms of goals and progressive carries:

*   **Jamie Gittens:** With 8 goals and a remarkable 113 progressive carries in 32 matches (1776 minutes), Gittens demonstrates a strong ability to both score and drive the ball forward. His xG of 3.6 suggests he is also getting into good scoring positions consistently.
*   **Richarlison:** With 4 goals and 8 progressive carries in 15 matches (504 minutes), Richarlison demonstrates a good ability to both score and drive the ball forward. His xG of 3.7 suggests he is also getting into good sc

'Based on the provided scouting reports, the following forwards stand out in terms of goals and progressive carries:\n\n*   **Jamie Gittens:** With 8 goals and a remarkable 113 progressive carries in 32 matches (1776 minutes), Gittens demonstrates a strong ability to both score and drive the ball forward. His xG of 3.6 suggests he is also getting into good scoring positions consistently.\n*   **Richarlison:** With 4 goals and 8 progressive carries in 15 matches (504 minutes), Richarlison demonstrates a good ability to both score and drive the ball forward. His xG of 3.7 suggests he is also getting into good scoring positions consistently.\n*   **Joel Pohjanpalo:** With 6 goals and 9 progressive carries in 20 matches (1603 minutes), Pohjanpalo demonstrates a good ability to both score and drive the ball forward. His xG of 4.3 suggests he is also getting into good scoring positions consistently.\n*   **Giorgos Masouras:** With 2 goals and 26 progressive carries in 14 matches (1055 minute

In [ ]:

df_clean.to_csv("data/players24_25.csv", index=False)
print(f"Sparat {len(df_clean)} spelare till data/players24_25.csv")
print("\nNu kan du köra Streamlit-appen med:")
print("streamlit run app.py")

Sparat 2328 spelare till data/players24_25.csv

Nu kan du köra Streamlit-appen med:
streamlit run app.py
